In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split


2019-02-26 20:38:05,876 utils 340 [INFO]    [logger_func] start 


In [109]:
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg', 'type']

base = utils.read_pkl_gzip('../input/base_type_group.gz')[[key, target, 'type']]
base["rank"] = base['type'].map(lambda x: 
                                1 if x==0 else
                                2 if x==1 else
                                3 if x==2 else
                                4
                               )

#========================================================================
# 
win_path_list = glob.glob('../model/LB3667_single_70leaves/*.gz')
base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)

feature_list = utils.parallel_load_data(path_list=win_path_list)
df_feat = pd.concat(feature_list, axis=1)

train = pd.concat([base_train, df_feat.iloc[:len(base_train), :]], axis=1)
test = pd.concat([base_test, df_feat.iloc[len(base_train):, :].reset_index(drop=True)], axis=1)
train[target] = base[target].map(lambda x: 1 if x<-30 else 0)
#========================================================================

# train = train.head(3000)
# train = base[~base[target].isnull()].head(30000)
train.fillna(-1, inplace=True)
# test = base[~base[target].isnull()]

use_cols = [col for col in train.columns if col not in ignore_list and not(str(type(train[col].values[0])).count('date'))  and not(str(type(train[col].values[0])).count('str')) ]
x_train, x_val, y_train, y_val = train_test_split(train[use_cols], train[target], test_size=0.2)

In [110]:
x_train.sort_values(by='rank', inplace=True)
trn_idx = x_train.index
y_train = y_train.loc[trn_idx]

x_val.sort_values(by='rank', inplace=True)
trn_idx = x_val.index
y_val = y_val.loc[trn_idx]


import lightgbm as lgb
num_boost_round = 100
early_stopping_rounds = 100

metric = 'ndcg'
metric = 'rmse'
params = {}
params['metric'] = metric
#  params['query'] = -1
# params['query'] = "name:rank"
params['query'] = "rank"
params['task'] = 'train'
objective = 'lambdarank'
objective = 'regression'
params['objective'] = objective
# params['ndcg_eval_at'] = [1]  # for lambdarank
# max_position = 2
# params['max_position'] = max_position  # for lambdarank


# rank file
rank_cnt = 0
query_list = []
for num, rank in enumerate(x_train['rank'].values):
    if rank_cnt==0:
        rank_cnt+=1
        before_rank = rank
    elif before_rank==rank:
        rank_cnt+=1
    else:
        query_list.append(rank_cnt)
        rank_cnt=1
        before_rank = rank
else:
    query_list.append(rank_cnt)
    
query = pd.Series(query_list)
print(np.sum(query))
print(x_train.shape)
query.to_csv('../../../tool/LightGBM/examples/lambdarank/rank.train.query', index=False)

rank_cnt = 0
query_list = []
for rank in x_val['rank'].values:
    if rank_cnt==0:
        rank_cnt+=1
        before_rank = rank
    elif before_rank==rank:
        rank_cnt+=1
    else:
        query_list.append(rank_cnt)
        rank_cnt=1
        before_rank = rank
else:
    query_list.append(rank_cnt)
        
pd.Series(query_list).to_csv('../../../tool/LightGBM/examples/lambdarank/rank.test.query', index=False)
print(np.sum(query))
print(x_train.shape)

save_cols = [col for col in use_cols if col!='rank' and col!='type']
x_train[save_cols].to_csv('../../../tool/LightGBM/examples/lambdarank/rank.train', index=False, header=False)
x_val[save_cols].to_csv('../../../tool/LightGBM/examples/lambdarank/rank.test', index=False, header=False)
sys.exit()
    

# Dataset
lgb_train = lgb.Dataset(data=x_train, label=y_train)
lgb_eval = lgb.Dataset(data=x_val, label=y_val)

#  self.logger.info(params)
estimator = lgb.train(
    train_set=lgb_train,
    valid_sets=lgb_eval,
    params=params,
    verbose_eval=100,
    early_stopping_rounds=early_stopping_rounds,
    num_boost_round=num_boost_round,
    categorical_feature= []
)

161533
(161533, 299)
161533
(161533, 299)


SystemExit: 

In [77]:
x_train

,hist_personal_term,new_personal_term,hist_regist_term,new_regist_term,clf_pred,type,rank
13292,10,1.0,24,24.0,0.081051,0,1
6081,6,2.0,6,8.0,0.003060,0,1
27112,8,2.0,8,13.0,0.003593,0,1
12392,9,2.0,21,18.0,0.014724,0,1
6114,6,1.0,6,8.0,0.005767,0,1
18062,6,1.0,7,14.0,0.008079,0,1
15030,5,1.0,5,7.0,0.001807,0,1
19380,12,1.0,17,18.0,0.011415,0,1
6344,8,2.0,8,11.0,0.015208,0,1
26797,8,2.0,8,10.0,0.004284,0,1
